Сейчас мое ядро выглядит таким образом:

In [ ]:
import pandas as pd
import re
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Load in the train and test datasets
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

# Store our passenger ID for easy access
PassengerId = test['PassengerId']

train.head(3)

In [ ]:
full_data = [train, test]

# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

In [ ]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [ ]:
train.head(3)

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

Итак, у нас получились два датасета с новыми признаками. Теперь приступим к построению модели.

### Построение модели

### 1.

Воспользуюсь алгоритмом стекинга. В качестве базовых алгоритмов использую RandomForestClassifier, SVC, GradientBoostingClassifier и LinearRegression; в качестве мета-алгоритма - XGBoost.

Разделю данные train на тренировочную и валидационную выборки с random_state=17 и параметром разбиения test_size=.3 (в качестве целевой переменной возьму столбец Survived, а в качестве признаков - все остальные столбцы).

Ниже приведены параметры для каждого из базовых алгоритмов, которые необходимо настроить на 5-кратной кросс-валидации с помощью GridSearchCV:

In [ ]:
from sklearn.model_selection import (GridSearchCV,
                                     train_test_split,
                                     StratifiedKFold)

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

# параметры базовых алгоритмов
gbc_params = {'learning_rate': np.arange(0.1, 0.6, 0.1)} # GradientBoostingClassifier

rfc_params = {'n_estimators': range(10, 100, 10), # RandomForestClassifier
              'min_samples_leaf': range(1, 5)}

svc_params = {'kernel': ['linear', 'rbf'], # SVC
              'C': np.arange(0.1, 1, 0.2),
             'gamma': np.logspace(-5, 2, num=8), 
              'degree': np.arange(2, 8, 1)}

lr_params = {'C': np.arange(0.5, 1, 0.1),
            'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

skf = StratifiedKFold(n_splits=5, random_state=17)

In [ ]:
targets = train.Survived
data    = train.drop(columns='Survived')

x_train, x_test, y_train, y_test = train_test_split(data, 
                                                    targets,
                                                    test_size=0.3,
                                                    random_state=17)

rfc = RandomForestClassifier(random_state=17)
svc = SVC(random_state=17)
gbc = GradientBoostingClassifier(random_state=17)
lr  = LogisticRegression(random_state=17)

meta = XGBClassifier()

### 2.
1. Определю объект GridSearchCV для всех приведенных параметров каждого алгоритма (в гиперпараметрах алгоритма при его определении, укажу random_state=17). Параметр cv устанавливаю равным skf.

2. Обучу каждый из объектов из 1-го пункта на получившейся при разбиении тренировочной выборке. Выведу лучшее сочетание параметров для каждого из алгоритмов.

3. Для каждого обученного алгоритма получу предсказания на валидационных данных и выведу метрику качества, которая соответствует метрике оценки соревнования.

In [ ]:
# Определю объект GridSearchCV для всех приведенных параметров каждого алгоритма (в гиперпараметрах алгоритма при 
# его определении, укажу random_state=17). Параметр cv устанавливаю равным skf.

rfc_model = GridSearchCV(rfc, rfc_params, cv=skf)
svc_model = GridSearchCV(svc, svc_params, cv=skf)
gbc_model = GridSearchCV(gbc, gbc_params, cv=skf)
lr_model = GridSearchCV(lr, lr_params, cv=skf)

In [ ]:
# 2.Обучу каждый из объектов из 1-го пункта на получившейся при разбиении тренировочной выборке.
rfc_model_fit = rfc_model.fit(x_train, y_train)
svc_model_fit = svc_model.fit(x_train, y_train)
gbc_model_fit = gbc_model.fit(x_train, y_train)
lr_model_fit = lr_model.fit(x_train, y_train)

# Выведу лучшее сочетание параметров для каждого из алгоритмов
rfc_best_param = rfc_model.best_params_
print(f'Лучшее сочетание параметров для алгоритма RandomForestClassifier = {rfc_best_param}')

svc_best_param = svc_model.best_params_
print(f'Лучшее сочетание параметров для алгоритма SVC = {svc_best_param}')

gbc_best_param = gbc_model.best_params_
print(f'Лучшее сочетание параметров для алгоритма GradientBoostingClassifier = {gbc_best_param}')

lr_best_param = lr_model.best_params_
print(f'Лучшее сочетание параметров для алгоритма LogisticRegression = {lr_best_param}')

In [ ]:
# 3. 
rfc_pred = rfc_model.predict(x_test)
svc_pred = svc_model.predict(x_test)
gbc_pred = gbc_model.predict(x_test)
lr_pred = lr_model.predict(x_test)

accuracy_rfc = rfc_model.score(x_test, y_test)
print(f'Accuracy rfc: {accuracy_rfc}')

accuracy_svc = svc_model.score(x_test, y_test)
print(f'Accuracy svc: {accuracy_svc}')

accuracy_gbc = gbc_model.score(x_test, y_test)
print(f'Accuracy gbc: {accuracy_gbc}')

accuracy_lr = lr_model.score(x_test, y_test)
print(f'Accuracy lr: {accuracy_lr}')

In [ ]:
from sklearn.metrics import roc_auc_score

print(f'Accuracy rfc : {roc_auc_score(y_test, rfc_pred)}')
print(f'Accuracy svc : {roc_auc_score(y_test, svc_pred)}')
print(f'Accuracy gbc : {roc_auc_score(y_test, gbc_pred)}')
print(f'Accuracy lr : {roc_auc_score(y_test, lr_pred)}')

### 3.
С помощью GridSearchCV и указанных ниже параметров настрою мета-алгоритм на мета-признаках (использую 5-кратную валидацию и random_state=17 при определении алгоритма). Матрицу метапризнаков получу из предсказаний, полученных в предыдущем пункте на валидационных данных базовыми алгоритмами. Выведу лучшие параметры.

In [ ]:
xgb_params = {'n_estimators': range(10, 100, 5),
              'eta': np.arange(0.1, 1., .1),
              'min_child_weight': range(1, 10, 1),
              'subsample': np.arange(0.1, 1., 0.2)}

In [ ]:
xgb = GridSearchCV(meta, xgb_params, cv=skf)

In [ ]:
models_pred = np.array([rfc_pred, svc_pred, gbc_pred, lr_pred])

meta_mtrx = np.empty((y_test.shape[0], len(models_pred)))
for n, model in enumerate(models_pred):
    meta_mtrx[:, n] = model
    
xgb.fit(meta_mtrx, y_test)

In [ ]:
# Выведу лучшие параметры.
xgb.best_params_

### 4.
Построю стекинг (использую 5-кратную кросс-валидацию) для всех моделей с наилучшими подобранными параметрами. В качестве тренировочных данных использую весь датасет train.csv, а в качестве тестовых - весь датасет test.csv. Сделаю прогноз мета-алгоритма для test.csv.

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, random_state=None, test_size=None, cv=5):
    
    if test_size is None:
        
        meta_mtrx = np.empty((data_train.shape[0], len(models)))

        for n, model in enumerate(models):
            meta_mtrx[:, n] = cross_val_predict(model, data_train, targets_train, cv=cv, method='predict')
            model.fit(data_train, targets_train)

        meta_alg.fit(meta_mtrx, targets_train)

        meta_mtrx_test = np.empty((data_test.shape[0], len(models)))

        for n, model in enumerate(models):
            meta_mtrx_test[:, n] = model.predict(data_test)

    elif test_size > 0 and test_size < 1:
        
        x_train, x_test, y_train, y_test = train_test_split(data_train, targets_train, test_size=test_size, random_state=random_state)
        
        meta_mtrx = np.empty((x_test.shape[0], len(models)))

        for n, model in enumerate(models):
            model.fit(x_train, y_train)
            meta_mtrx[:, n] = model.predict(x_test)

        meta_mtrx_pred = meta_alg.fit(meta_mtrx, y_test)

        meta_mtrx_test = np.empty((data_test.shape[0], len(models)))

        for n, model in enumerate(models):
            meta_mtrx_test[:, n] = model.predict(data_test)

    else:
        raise ValueError("test_size must be between 0 and 1")
        
    return meta_alg, meta_mtrx_test

In [ ]:
rfc = RandomForestClassifier(min_samples_leaf=3, n_estimators=60, random_state=17)
svc = SVC(C=0.1, degree=2, gamma=1e-05, kernel='linear', random_state=17)
gbc = GradientBoostingClassifier(learning_rate=0.1, random_state=17)
lr = LogisticRegression(C=0.8999999999999999, solver='lbfgs', random_state=17)

xgb = XGBClassifier(eta=0.30000000000000004, min_child_weight=1, n_estimators=20, subsample=0.1)

models = [rfc, lr, svc, gbc] # список базовых алгоритмов с наилучшими подобранными параметрами

In [ ]:
# тест на тренировочных данных
meta_alg, meta_mtrx_test = stacking(models = models, 
                                    meta_alg = xgb, 
                                    data_train = x_train,
                                    targets_train = y_train,
                                    data_test = x_test,
                                    targets_test = y_test, 
                                    test_size = 0.3)

In [ ]:
meta_alg_predict = meta_alg.predict(meta_mtrx_test)

In [ ]:
print(f'Stacking AUC: {roc_auc_score(y_test, meta_alg_predict)}')

In [ ]:
# В качестве тренировочных данных использую весь датасет train.csv, а в качестве тестовых - весь датасет test.csv. 
# Сделаю прогноз мета-алгоритма для test.csv.
meta_alg, meta_mtrx_test = stacking(models = models, 
                                    meta_alg = xgb,
                                    data_train = data,
                                    targets_train = targets,
                                    data_test = test,
                                    targets_test = None, 
                                    test_size = 0.3)

### 5.
С помощью нижеприведенной функции сформирую файл посылки для соревнования и отправлю на Kaggle.

In [ ]:
def write_to_submission_file(meta_predictions, PassengerID, out_file='Submission.csv', columns=['PassengerID', 'Survived']):
    predicted_df = pd.DataFrame(np.array([PassengerId, meta_predictions]).T, columns=columns)
    predicted_df.to_csv(out_file, index=False)

In [ ]:
# ваш код
predictions = meta_alg.predict(meta_mtrx_test)
PassengerID = PassengerId

write_to_submission_file(predictions, PassengerID)

In [ ]:
predictions

### 6.
Результат score, полученного на соревновании

Ваш ответ: 0.79425